### **Q_1**

In [0]:
import pandas as pd

countries = pd.read_csv("/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/2_silver/silver_finance/silver_countries.csv")
countries = countries[['name', 'alpha-2', 'alpha-3']]
countries = countries.rename(columns={'name' : 'country-name'})

ikea_stores = pd.read_csv("/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/2_silver/silver_stores/silver_stores.csv")
ikea_stores['alpha-2'] = ikea_stores['locale'].str[:2]
ikea_stores['alpha-2'] = ikea_stores['alpha-2'].str.upper()
ikea_stores = ikea_stores[['id', 'alpha-2']]
ikea_stores = ikea_stores.drop_duplicates()

pivot_table = pd.merge(ikea_stores, countries, on='alpha-2', how='inner')
pivot_table = pivot_table.sort_values(by='id')

counts = pivot_table.groupby(['country-name', 'alpha-2', 'alpha-3']).size().reset_index(name='sum_of_IKEA_stores')
display(counts)
counts.to_parquet('/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/3_gold/Q_1_pivot_table_sum_IKEA_stores.parquet', engine='pyarrow')

### **Q_2**

In [0]:
big_mac = pd.read_csv("/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/2_silver/silver_economist/silver_big-mac-data.csv")
#big_mac = big_mac.sort_values(by=['name', 'date'], ascending=[True, False])

big_mac['date'] = pd.to_datetime(big_mac['date'])
big_mac = big_mac[big_mac['date'] >= pd.to_datetime('today') - pd.DateOffset(years=5)]
big_mac['date'] = big_mac['date'].dt.strftime('%Y-%m-%d')
big_mac = big_mac.rename(columns={'iso_a3' : 'alpha-3'})

merge_table = pd.merge(big_mac, counts, on='alpha-3', how='inner')
#unique_countries_count = merge_table['name'].nunique()
#print(unique_countries_count)


# Find countries in df2 but not in df1
#countries_in_df2_not_in_df1 = counts[~counts['alpha-3'].isin(merge_table['alpha-3'])]
#display(countries_in_df2_not_in_df1)

import numpy as np
merge_table['price_USD'] = np.where(merge_table['dollar_ex'] != 0, merge_table['local_price'] / merge_table['dollar_ex'], np.nan).round(2)

merge_table = merge_table[['name', 'local_price', 'price_USD','date']]
merge_table = merge_table.drop_duplicates()

table_5y_avg_price = merge_table.groupby('name')[['local_price', 'price_USD']].mean().round(2).reset_index()

display(table_5y_avg_price)
table_5y_avg_price.to_parquet('/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/3_gold/Q_2_table_5y_avg_price.parquet', engine='pyarrow')

### **Q_3**

In [0]:
merge_table['date'] = pd.to_datetime(merge_table['date'])
latest_price = merge_table.sort_values(by=['name', 'date'], ascending=[True, False])
latest_price = latest_price.drop_duplicates(subset='name', keep='first')
latest_price = latest_price[['name', 'local_price', 'price_USD']]
display(latest_price)
latest_price.to_parquet('/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/3_gold/Q_3_latest_price.parquet', engine='pyarrow')

In [0]:

# Specify the file path to your Parquet file
file_path = '/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/3_gold/latest_price.parquet'

# Read the Parquet file into a DataFrame
df = pd.read_parquet(file_path, engine='pyarrow')

# Display the DataFrame
display(df)

### **Big-mac-silver**

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

input_path = "/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/1_bronze/src_economist"
output_path = "/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/2_silver/silver_economist"
file_name = "big-mac-data"

batch_paths = [
    f"{input_path}/batch_1/big-mac-source-data-v2_1.csv",
    f"{input_path}/batch_2/big-mac-source-data-v2_2.csv",
    f"{input_path}/batch_3/big-mac-source-data-v2_3.csv",
]
spark = SparkSession.builder.appName("Example").getOrCreate()

schema = StructType([
    StructField("name", StringType(), True),
    StructField("iso_a3", StringType(), True),
    StructField("currency_code", StringType(), True),
    StructField("local_price", DoubleType(), True),
    StructField("dollar_ex", DoubleType(), True),
    StructField("GDP_dollar", DoubleType(), True),
    StructField("GDP_local", DoubleType(), True),
    StructField("date", StringType(), True)
])

silver_big_mac = spark.createDataFrame(data=[], schema=schema)

for path in batch_paths:
    #raw_data = spark.read.format("csv").option("header", "true").load(path)
    raw_data = pd.read_csv(path)

    # Convert Pandas DataFrame to PySpark DataFrame
    raw_data = spark.createDataFrame(raw_data)

    # Apply transformations
    cleaned_data = raw_data.dropDuplicates()
    # Write to Silver layer
    #cleaned_data.toPandas().to_csv(f"{output_path}/silver_{file_name}.csv")
    #cleaned_data.write.mode("append").option("header", "true").csv(f"{output_path}/silver_{file_name}.csv")
    silver_big_mac = silver_big_mac.union(cleaned_data)

silver_big_mac.toPandas().to_csv(f"{output_path}/silver_{file_name}.csv")


### **Stores-bronze JSON**

In [0]:
import requests
import json

# Step 1: Fetch data from the web
url = "https://www.ikea.com/global/assets/informera/stores/stores-unfiltered-detailed.json"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Step 2: Parse the response into JSON
    data = response.json()
    
    # Step 3: Save the JSON file in Databricks File System (DBFS)
    json_path = "/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/1_bronze/src_stores/bronze_stores.json"  # Adjust the path as needed
    
    with open(json_path, "w") as json_file:
        json.dump(data, json_file, indent=4)
    
    print(f"JSON file saved successfully at {json_path}")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")


### **Stores-silver**

In [0]:
import pandas as pd
from pandas import json_normalize

# Step 1: Load the JSON file into a pandas DataFrame
input_path = "/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/1_bronze/src_stores/bronze_stores.json"
output_path = "/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/2_silver/silver_stores"
file_name = "stores"

data = pd.read_json(input_path)

# Flatten the locales data
flattened_data = []
for index, row in data.iterrows():
    id, lat, lng, locales = row["id"], row["lat"], row["lng"], row["locales"]
    for locale_key, locale_value in locales.items():
        entry = {
            "id": id,
            "lat": lat,
            "lng": lng,
            "locale": locale_key,
            "displayName": locale_value["displayName"],
            "displayNameAlternate": locale_value["displayNameAlternate"],
            "street": locale_value["address"].get("street"),
            "zipCode": locale_value["address"].get("zipCode"),
            "city": locale_value["address"].get("city"),
            "timezone": locale_value["address"].get("timezone"),
            "stateProvinceCode": locale_value["address"].get("stateProvinceCode"),
            "displayAddress": locale_value["address"].get("displayAddress"),
        }
        flattened_data.append(entry)

# Create a new DataFrame from the flattened data
df_flattened = pd.DataFrame(flattened_data)

# Step 3: Drop duplicates
silver_stores = df_flattened.drop_duplicates()

# Step 4: Save as CSV
silver_stores.to_csv(f"{output_path}/silver_{file_name}.csv", index=False)

print(f"CSV file saved successfully at {output_path}/silver_{file_name}.csv")

### **Display data**

In [0]:
# Import required library
import pandas as pd

# Read the CSV file
df = pd.read_csv("/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/2_silver/silver_stores/silver_stores.csv")

# Display the first few rows
df.head()


In [0]:
# Import required library
import pandas as pd

# Read the CSV file
df = pd.read_csv("/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/data_storage/2_silver/silver_finance/silver_countries.csv")

# Display the first few rows
df.head()